In [86]:
%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_autonomia_economica.ipynb
#%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/02_funciones_autonomia_economica.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings('ignore')

In [87]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [88]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [89]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [90]:
tasa, size, ratio, error = poblacion_inactiva_sin_ingresos(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
dependientes_sin_ingreso = pd.concat([tasa, size, ratio, error], axis=1)

In [91]:
dependientes_sin_ingreso.to_pickle(pathdata + 'data_output/df_dependientes_sin_ingresos_pool.pickle')

In [92]:
df_temp = df_people_pool.query('CH06 >= 14')

bins = [14, 30, 46, 66, df_temp['CH06'].max()]
labels = ['14-29', '30-45', '46-65', '65+']

# Agregar una columna con los rangos de edad
df_temp['Rango edad'] = pd.cut(df_temp['CH06'], bins=bins, labels=labels, right=False)

pondera = 'PONDERA_new'

df_inactivos_con_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp['P47T']==0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].sum().unstack(level=0)
df_inactivos_sin_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp['P47T']>0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].sum().unstack(level=0)
df_inactivos_no_declara_ingreso = df_temp[(df_temp['ESTADO']==3) & (df_temp['P47T']==-9) & (df_temp['CAT_INAC']!=3)].groupby(['CH04','Rango edad'])[pondera].sum().unstack(level=0)


In [93]:
df_inactivos_con_ingreso.columns = pd.MultiIndex.from_tuples([('Ingreso > 0', 'Varon'), ('Ingreso > 0', 'Mujer')])
df_inactivos_sin_ingreso.columns = pd.MultiIndex.from_tuples([('Ingreso = 0', 'Varon'), ('Ingreso = 0', 'Mujer')])
df_inactivos_no_declara_ingreso.columns = pd.MultiIndex.from_tuples([('Ingreso = -9', 'Varon'), ('Ingreso = -9', 'Mujer')])


In [94]:
df_inactivos = pd.concat([df_inactivos_con_ingreso, df_inactivos_sin_ingreso, df_inactivos_no_declara_ingreso], axis=1)
df_inactivos.loc['Total Col'] = df_inactivos.sum(numeric_only=True, axis=0)
tabla_pob_inactiva = df_inactivos.style.format('{:,.0f}')

In [95]:
df_inactivos_porcentaje = df_inactivos.iloc[:-1,:] / df_inactivos.iloc[-1,:] * 100
df_inactivos_porcentaje.loc['Total Col'] = df_inactivos_porcentaje.sum(numeric_only=True, axis=0)
tabla_pob_inactiva_porcentaje = df_inactivos_porcentaje.style.format('{:,.2f}')

In [96]:
tabla_pob_inactiva

In [97]:
tabla_pob_inactiva_porcentaje

In [108]:
ratio_pob_inactiva = pd.DataFrame()

In [109]:
ratio_pob_inactiva.loc[:,'Ingreso > 0'] = df_inactivos_porcentaje.iloc[:,1]/df_inactivos_porcentaje.iloc[:,0] * 100
ratio_pob_inactiva.loc[:,'Ingreso = 0'] = df_inactivos_porcentaje.iloc[:,3]/df_inactivos_porcentaje.iloc[:,2] * 100
ratio_pob_inactiva.loc[:,'Ingreso = -9'] = df_inactivos_porcentaje.iloc[:,5]/df_inactivos_porcentaje.iloc[:,4] * 100

In [111]:
ratio_pob_inactiva.drop(ratio_pob_inactiva.tail(1).index,inplace=True)


In [112]:
ratio_pob_inactiva

,Ingreso > 0,Ingreso = 0,Ingreso = -9
Rango edad,,,
14-29,43.083401,213.478580,457.065866
30-45,221.641890,309.725462,216.624886
46-65,156.106555,110.884525,113.888920
65+,84.359985,75.180247,88.501539


In [8]:
# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

In [9]:
dependientes_sin_ingreso.index = dependientes_sin_ingreso.index.map(dict_cod_provincia)
dependientes_sin_ingreso


,Varon,Mujer,N_v,N_m,Dependencia,LI,LS,ME,ER
PROVINCIA,,,,,,,,,
CABA,3.161835,5.973103,31,66,52.934552,44.530314,61.338790,8.404238,15.876658
Buenos Aires,3.529948,11.892641,239,783,29.681783,26.423846,32.939721,3.257937,10.976218
Catamarca,5.207041,8.325888,93,155,62.540365,56.271010,68.809720,6.269355,10.024493
Córdoba,2.809257,8.736175,91,271,32.156608,27.378786,36.934430,4.777822,14.857979
Corrientes,4.643031,11.239638,81,174,41.309435,34.750281,47.868590,6.559154,15.878102
Chaco,5.301377,13.742870,73,209,38.575474,31.632377,45.518572,6.943097,17.998735
Chubut,4.072883,14.195559,85,269,28.691251,22.770828,34.611675,5.920424,20.634945
Entre Ríos,3.416503,11.983981,109,310,28.508915,23.539008,33.478822,4.969907,17.432818
Formosa,7.372137,13.745582,114,241,53.632772,47.159067,60.106476,6.473705,12.070427


In [10]:
import plotly.graph_objects as go

df_temp = dependientes_sin_ingreso.copy()
df_temp.sort_values(by='Dependencia', ascending=False, inplace=True)

bar_trace = go.Bar(
    x=df_temp.index,
    y=df_temp['Dependencia'],
    name='Autonomia economica',
    marker_color=fundar_colores_primarios[3],
    hovertemplate='Dependencia: %{y:.2f}<extra></extra>'
)

layout = go.Layout(
    title='Indicador dependientes sin ingreso',
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6, y=1.1, orientation='h'),
    yaxis_title="Ratio"

)

fig1 = go.Figure(data=[bar_trace], layout=layout)

fig1.add_shape(
    type='line',
    x0=df_temp.index[0],
    y0=100,
    x1=df_temp.index[-1],
    y1=100,
    line=dict(
        color='black',
        width=1,
        dash='dash'
    ),
)

fig1.update_yaxes(range=[0, 100])
file_path = pathdata + 'figs/'
filename = 'panel-dependientes-sin-ingreso-pool.html'
fig1.write_html(file_path + filename)

